In [13]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [10]:
pc = Pinecone()
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

In [12]:
import json
data = json.load(open("reviews.json"))

In [16]:
processed_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values" : embedding,
        "id" : review["professor"],
        "metadata": {
            "review": review["review"],
            "subject" : review["subject"],
            "stars" : review["stars"],
        }
    })

In [ ]:
processed_data[0]

In [18]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace='ns1',
       
)

{'upserted_count': 20}

In [19]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}